In [1]:
#installing kaggle
!pip install kaggle

Map -Charting Students Math Misunderstandings. Goal:to predict top 3 Misconceptions for student's math responds. Used DeBERta Hugging face NLP model to train model +inference +submission.csv.
Metric-MAP@3

The Model separates content and position and understands meaning without confusing order of words with semantic meaning making a strong training model.

upload kaggle.json file before this

In [2]:
#configuring a path for json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle competitions download -c map-charting-student-math-misunderstandings


  0% 0.00/815k [00:00<?, ?B/s]
100% 815k/815k [00:00<00:00, 974MB/s]


In [4]:
#extracting  zip file
from zipfile import ZipFile
dataset ='/content/map-charting-student-math-misunderstandings.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('the data set is extracted')




the data set is extracted


datasets : train.csv
test.csv
57 classes

requirements.txt

torch==2.2.0
transformers==5.5.0
pandas==2.1.0
numpy==1.27.0
scikit-learn==1.3.2
python 3

In [5]:
#libraries
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"




In [6]:
!pip install transformers datasets accelerate scikit-learn torch --quiet
!pip install evaluate

##libraries used
import pandas as pd
import numpy as np
import torch
import sys
import importlib,transformers
importlib.reload(transformers)
import joblib
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns

#data and machine learning
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer,Trainer,TrainingArguments)
from torch.utils.data import DataLoader
from evaluate import load
from huggingface_hub import snapshot_download
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00


In [7]:
##load data
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [8]:
# 6 Check data quality
print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("Null values in train:", train.isnull().sum())
print("Null values in test:", test.isnull().sum())

Train shape: (36696, 7)
Test shape: (3, 5)
Null values in train: row_id                    0
QuestionId                0
QuestionText              0
MC_Answer                 0
StudentExplanation        0
Category                  0
Misconception         26836
dtype: int64
Null values in test: row_id                0
QuestionId            0
QuestionText          0
MC_Answer             0
StudentExplanation    0
dtype: int64


In [9]:
# 7. Handle null values in Misconception properly
print(f"Missing values in Misconception: {train['Misconception'].isnull().sum()} / {len(train)}")

# Create target variable handling nulls
train['target'] = train.apply(
    lambda x: f"{x['Category']}:{x['Misconception']}" if pd.notna(x['Misconception']) else f"{x['Category']}:NA",
    axis=1
)


Missing values in Misconception: 26836 / 36696


In [10]:
# 8. Check class distribution
class_counts = train['target'].value_counts()
print("Original classes:", len(class_counts))
print("Top 10 classes:")
print(class_counts.head(10))

Original classes: 65
Top 10 classes:
target
True_Correct:NA                       14802
False_Neither:NA                       6542
True_Neither:NA                        5265
False_Misconception:Incomplete         1446
False_Misconception:Additive            891
False_Misconception:Duplication         698
False_Misconception:Subtraction         618
False_Misconception:Positive            564
False_Misconception:Wrong_term          550
False_Misconception:Wrong_fraction      412
Name: count, dtype: int64


preprocessing
tokenizing using Deberta tokenizer ,max_length=128
weighting for the imbalanced classes applied during  loss computation
labels ended using the label encoder

In [11]:
# 9. Remove rare classes (keep only meaningful ones)
MIN_SAMPLES = 20  # Increased for better training
valid_classes = class_counts[class_counts >= MIN_SAMPLES].index
train_clean = train[train['target'].isin(valid_classes)].copy()

print(f"After cleaning: {len(train_clean)} samples (from {len(train)})")
print(f"Remaining classes: {len(valid_classes)}")

After cleaning: 36544 samples (from 36696)
Remaining classes: 38


In [12]:
# 10. Prepare text data
train_clean['text'] = (
    train_clean['QuestionText'] + " [SEP] Answer: " +
    train_clean['MC_Answer'] + " [SEP] Explanation: " +
    train_clean['StudentExplanation']
)
test['text'] = (
    test['QuestionText'] + " [SEP] Answer: " +
    test['MC_Answer'] + " [SEP] Explanation: " +
    test['StudentExplanation']
)

In [13]:
# 11. Encode labels
le = LabelEncoder()
train_clean['labels'] = le.fit_transform(train_clean['target'])

print(f"Final training samples: {len(train_clean)}")
print(f"Number of classes: {len(le.classes_)}")


Final training samples: 36544
Number of classes: 38


In [14]:
# 12. Create datasets
dataset = Dataset.from_pandas(train_clean[['text', 'labels']])
train_val = dataset.train_test_split(test_size=0.1, seed=42)

In [15]:
# 13. Tokenize
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small")
def tokenize_function(batch):
    return tokenizer(batch['text'], truncation=True, padding=True, max_length=256)

train_val = train_val.map(tokenize_function, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/32889 [00:00<?, ? examples/s]

Map:   0%|          | 0/3655 [00:00<?, ? examples/s]

In [16]:
# 14. Initialize model
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-small",
    num_labels=len(le.classes_)
)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# 15. Training setup
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_val['train'],
    eval_dataset=train_val['test'],
    tokenizer=tokenizer,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-790334210.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
# 16. Train
print("Starting training...")
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Starting training...


Epoch,Training Loss,Validation Loss
1,0.560900,0.527826
2,0.490700,0.431673
3,0.371400,0.432775


TrainOutput(global_step=12336, training_loss=0.6150495834387694, metrics={'train_runtime': 2638.2203, 'train_samples_per_second': 37.399, 'train_steps_per_second': 4.676, 'total_flos': 4987029062424684.0, 'train_loss': 0.6150495834387694, 'epoch': 3.0})

In [20]:
# 17. Prepare test data
test_dataset = Dataset.from_pandas(test[['text']])
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [21]:
# 18. Predict
predictions = trainer.predict(test_dataset)
logits = predictions.predictions


In [22]:
# 19. Get top 3 predictions
top3_preds = []
for logit in logits:
    top3_idx = np.argsort(logit)[-3:][::-1]
    top3_labels = [le.inverse_transform([idx])[0] for idx in top3_idx]
    top3_preds.append(" ".join(top3_labels))

In [23]:
# 20. Create submission
submission = pd.DataFrame({
    'row_id': test['row_id'],
    'Category:Misconception': top3_preds
})
submission.to_csv("submission.csv", index=False)


hardware used was  GPU
Hugging Face Trainer for training and evaluation
validation loss of 0.96 reasonably good learning patterns
validation is 0.86 means the model is very sound and predict Category:Misconception correctly 86% of the time




MIT License

Copyright (c) 2025 Gloria

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND...
